In [6]:
PATH_TO_VIDEO = "/home/ezaborshchikov/hse_mc/Masters_tethis/videos/VID_20250415_181952.mp4"

In [7]:
from ultralytics import YOLO
import cv2

# model = YOLO("models/yolov8n.pt")  # nano-версия (быстрая)
# video = "/home/ezaborshchikov/hse_mc/Masters_tethis/videos/VID_20250415_181952.mp4"
# results = model(video, classes=[2, 5, 7], stream=True)  # 2=авто, 5=автобус, 7=грузовик

# Load the YOLO11 model
model = YOLO("models/yolo11n.pt")

# Open the video file
# video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(PATH_TO_VIDEO)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

100%|██████████| 5.35M/5.35M [00:00<00:00, 10.4MB/s]
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00ta 0:00:01

requirements: AutoUpdate success ✅ 6.9s, installed 1 package: ['lap>=0.5.12']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 2 persons, 9 cars, 87.1ms
Speed: 4.3ms preprocess, 87.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 9 cars, 97.8ms
Speed: 37.0ms preprocess, 97.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 9 cars, 128.1ms
Speed: 9.9ms preprocess, 128.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 8 cars, 48.2ms
Speed: 13.1ms preprocess, 48.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 8 cars, 64.6ms